In [0]:
import numpy as np
import pandas as pd
import time

### Create synthetic dataset

In [0]:
n_rows = 1000000
n_cols = 500
n_clusters_data = 500
cluster_std = 1.0
# our transform code doesn't yet work with np.float32
#dtype='float32'
dtype='float64'
from cuml.datasets import make_blobs
data, _ = make_blobs(
        n_rows, n_cols, n_clusters_data, cluster_std=cluster_std, random_state=0, dtype=dtype
    )  # make_blobs creates a random dataset of isotropic gaussian blobs.

data = data.get()
data = 1000.0 * data

In [0]:
pd_data = pd.DataFrame({"features": list(data)})

In [0]:
df = spark.createDataFrame(pd_data)


### Run Cuml-on-Spark KMeans.fit

In [0]:
from sparkcuml.cluster import SparkCumlKMeans

In [0]:
num_workers = 2
n_clusters = 300
max_iter = 30

In [0]:
gpu_kmeans = SparkCumlKMeans(num_workers=num_workers,verbose=6)\
            .setK(n_clusters)\
            .setFeaturesCol("features")\
            .setMaxIter(max_iter)

KMeans estimator can be persisted and reloaded.

In [0]:
gpu_kmeans.write().overwrite().save("dbfs:/tmp/sparkcuml-kmeans-estimator")
gpu_kmeans_loaded = SparkCumlKMeans.load("dbfs:/tmp/sparkcuml-kmeans-estimator")


In [0]:
start_time = time.time()
gpu_model = gpu_kmeans_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 56.42046809196472 sec

In [0]:
gpu_kmeans_loaded.getOrDefault("n_clusters")

Out[10]: 300

In [0]:
[vec[0:10] for vec in gpu_model.cluster_centers_[0:2]]

Out[11]: [[4102.514858798706,
 -2507.8376546726035,
 449.4512890954289,
 -3549.3265354089413,
 2701.7209180093055,
 8087.1973588875335,
 -6658.842693416074,
 -3830.748262131907,
 4030.1612792966307,
 -2313.2967850788596],
 [587.5990000566869,
 4130.045814259852,
 745.8220404885038,
 77.4963683046966,
 -594.5914270994066,
 4621.595968330927,
 1986.4340070058395,
 -202.7495548968391,
 2187.5803931893834,
 2506.7198912227836]]

### Cuml-on-Spark KMeans model save, load, and transform

CPU cluster with 8 cores per exec

In [0]:
gpu_model.write().overwrite().save("dbfs:/tmp/sparkcuml-kmeans-model")

In [0]:
gpu_model_loaded = gpu_model.read().load("dbfs:/tmp/sparkcuml-kmeans-model")

In [0]:
[vec[0:10] for vec in gpu_model_loaded.cluster_centers_[0:2]]

Out[14]: [[4102.514858798706,
 -2507.8376546726035,
 449.4512890954289,
 -3549.3265354089413,
 2701.7209180093055,
 8087.1973588875335,
 -6658.842693416074,
 -3830.748262131907,
 4030.1612792966307,
 -2313.2967850788596],
 [587.5990000566869,
 4130.045814259852,
 745.8220404885038,
 77.4963683046966,
 -594.5914270994066,
 4621.595968330927,
 1986.4340070058395,
 -202.7495548968391,
 2187.5803931893834,
 2506.7198912227836]]

In [0]:
transformed_df = gpu_model_loaded.setPredictionCol("transformed").transform(df)

In [0]:
transformed_df.printSchema()

root
-- transformed: integer (nullable = false)

In [0]:
transformed_df.count()

Out[17]: 1000000

In [0]:
transformed_df.show(10)

+-----------+
transformed|
+-----------+
 42|
 146|
 231|
 290|
 14|
 17|
 2|
 287|
 213|
 93|
+-----------+
only showing top 10 rows

### Run PySpark ML KMeans.fit

Timed with 6 tasks per node parallelism, 2 nodes.

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
spark_ml_kmeans = KMeans()\
    .setFeaturesCol("features")\
    .setK(n_clusters)\
    .setMaxIter(max_iter)

Convert array sql type to VectorUDT expected by Spark ML algos

In [0]:
from pyspark.ml.functions import array_to_vector

In [0]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

In [0]:
start_time = time.time()
spark_ml_kmeans_model = spark_ml_kmeans.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 157.7422285079956 sec

In [0]:
type(spark_ml_kmeans_model.clusterCenters()[0])

Out[13]: numpy.ndarray

In [0]:
spark_ml_cluster_centers = spark_ml_kmeans_model.clusterCenters()
[vec[0:10] for vec in spark_ml_cluster_centers[0:2]]

Out[14]: [array([-8982.30849438, -5112.47453886, 5810.70505668, -6605.6783041 ,
 4239.5280306 , -8775.52589711, -5787.52248956, 4845.34171929,
 -3032.01636956, 2674.70256786]),
 array([-3633.3802357 , -4501.82431835, -3952.53439273, 1146.6177849 ,
 5478.41589899, 3613.93137942, 502.42421776, -3323.13336123,
 3634.96707952, 5595.04482085])]

In [0]:
spark_transformed = spark_ml_kmeans_model.setPredictionCol("transformed").transform(vector_df)

In [0]:
spark_transformed.filter(spark_transformed.transformed >= 0).count()

Out[16]: 1000000

In [0]:
spark_transformed.show(10)

+--------------------+-----------+
 features|transformed|
+--------------------+-----------+
[6404.27035218224...| 11|
[10650.4318461605...| 264|
[5380.86603679111...| 164|
[-3293.9581048535...| 216|
[2810.48663601147...| 115|
[10343.6019085224...| 293|
[-2020.2601910400...| 34|
[172.266256405292...| 152|
[10383.0326383522...| 272|
[-8316.4389961325...| 42|
+--------------------+-----------+
only showing top 10 rows

### Running cuml.KMeans.fit single-node on driver

In [0]:
from cuml import KMeans

In [0]:
cuml_kmeans = KMeans(n_clusters=n_clusters, max_iter=max_iter, verbose=4)

In [0]:
cuml_model = cuml_kmeans.fit(data)

In [0]:
cuml_model.n_iter_

Out[53]: 31

In [0]:
[vec[0:10] for vec in cuml_model.cluster_centers_[0:2]]

Out[54]: [array([ 9518.89506621, 7052.74798514, -6750.59047726, -5355.73139541,
 9647.54214206, -74.73821664, 4782.33284335, -3267.58683019,
 -9044.21698394, -7625.75893837]),
 array([ -167.4855979 , 137.499569 , -2034.28493232, 162.54515629,
 -7818.99917129, -3395.36444733, -2451.11175307, 6756.70686787,
 -65.16081362, -9550.56395252])]